<a href="https://colab.research.google.com/github/Reptilefury/coursera-machine-learning/blob/main/Introduction_To_Keras_Tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

In [ ]:
!pip install -q -U keras-tuner

     |████████████████████████████████| 133 kB 22.6 MB/s 


In [ ]:
import keras_tuner as kt 

Here we are going to use the keras tuner library to find the best hyperparameters that classifies images of clothing from the FashionMNIST dataset.

In [ ]:
#We load the data and normalize the pixel values to be in between zero and one
(train_images, train_labels),(test_images, test_labels) = tf.keras.datasets.fashion_mnist.load_data() 

4431872/4422102 [==============================] - 0s 0us/step


In [ ]:
#We normalize the dataset: standardize the dataset to be floating point numbers in between zero and one 
train_images = train_images.astype('float32')/255.0
test_images = test_images.astype('float32')/255.0

In [ ]:
#Check the first images and see whether it has been normalized
first_image = train_images[0]

In [ ]:
import numpy as np 
#check the maximum value
np.max(first_image)

3.637131e-15

In [ ]:
#This value is equivalent to 
3.63713 * 10-15 #one times ten to the power of 15 

21.371299999999998

In [ ]:
#check the minimum value
np.min(first_image)

0.0

When you build the model for hyper tuning we also define the hyperparameter search space in addition to the model architecture. The model you set up for hyper tuning is called a hyper model. 
We can define the hypermodel through two approaches:
- By using a model builder function.
- By subclassing the HyPerModel class of the Keras Tuner API.

We can also use two predefined model classes HyperXception and HyperResNet for computer vision applications.

In [ ]:
#Build the model 
def build_model(hp):
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Flatten(input_shape=train_images[0].shape))
  hp_units = hp.Int('units',min_value=32, max_value=512, step=32)
  model.add(tf.keras.layers.Dense(hp_units,activation='relu'))
  model.add(tf.keras.layers.Dense(10))


  #Tune the learning rate for the optimizer 
  hp_learning_rate = hp.Choice('learning_rate',values=[1e-2,1e-3,1e-4])
  model.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
                metrics = ['Accuracy']                
                )
  return model

In [ ]:
#create an instance of the model 
tuner = kt.Hyperband(build_model, objective = 'val_Accuracy', max_epochs=10, factor=3,directory='my_dir', project_name="intro_to_hypertuning")

INFO:tensorflow:Reloading Oracle from existing project my_dir/intro_to_hypertuning/oracle.json


In [ ]:
#Lets create a callback to stop training early when the validataion loss has reached a certain value
stop_early = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5)

In [ ]:
#The arguments for the search method are similar to that of the fit method 
tuner.search(train_images, train_labels, epochs = 50, validation_split = 0.2,callbacks=[stop_early])

Trial 30 Complete [00h 01m 31s]
val_Accuracy: 0.10300000011920929

Best val_Accuracy So Far: 0.10300000011920929
Total elapsed time: 00h 14m 01s
INFO:tensorflow:Oracle triggered exit


Hyperparameter tuning is the process of selecting the right set of hyperparameters for your model. Hyperparameters are parameters of the learning algorithm. 

In [2]:
!pip install -q -U keras-tuner

In [3]:
import keras_tuner as kt 

In [5]:
import tensorflow as tf
from tensorflow import keras

In [8]:
#In this tutorial we'll use keras tuner to find the best hyperparameters for a machine learning model that finds the best hyperparameters for a machine learning model 
#That classifiers images for clothing 
#We load the fashion mnist dataset 
(train_images,train_labels),(test_images,test_labels) = tf.keras.datasets.fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [11]:
#We standardize/normalize the data before passing it into our model to avoid biasness in our dataset, this increases the accuracy of training 
#The computations will be faster as our values will be floating point numbers in between zero and one
train_images = train_images.astype('float32')/255.0
test_images = test_images.astype('float32')/255.0

In [13]:
#Check whether the data has been normalized 
train_images_first = train_images[0]

In [16]:
import numpy as np 
np.max(train_images_first) #Maximum number in our array  
#This gives us 1.5 * 10 -5  This is a floating point number in between zero and one 

1.5378702e-05

In [20]:
input_shape = train_images_first.shape #Get the shape attribute of our input and pass it as a value to the input shape argument

In [37]:
def model_builder(hp):
  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.Flatten(input_shape=input_shape))
  hp_units = hp.Int('Units', min_value=32, max_value = 512, step = 32)
  model.add(tf.keras.layers.Dense(hp_units,activation='relu'))
  model.add(tf.keras.layers.Dense(10))

  #Tune the learning rate for the optimizer 
  #Choose the optimal value from between 0.01 , 0.001 and 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
  model.compile(loss = tf.keras.losses.BinaryCrossentropy(from_logits = True), optimizer = tf.keras.optimizers.Adam(learning_rate=hp_learning_rate), metrics =['Accuracy'])
  return model 